# Model Evaluation

In [91]:
import pandas as pd
import sklearn as skl

In [93]:
def loadData():
    # load data
    spam_data = pd.read_csv("spambase.csv")

    # shuffle data
    spam_data = spam_data.sample(frac=1)

    # first 80% is training, last 20% is test
    training = spam_data[:int(len(spam_data)*0.8)]
    test = spam_data[int(len(spam_data)*0.8):]

    # get kfold
    kfold = skl.model_selection.KFold(5)

    # loop over split values
    for train_index, valid_index in kfold.split(training):
        # get validation and training datasets
        sub_validation = training.iloc[valid_index]
        sub_training = training.iloc[train_index]

    return sub_validation, sub_training

In [94]:
def naiveBayes(training, test):

    pSpam = dict()
    pNot = dict()

    # loop over all columns (except the last 5)
    for col in range (54):
        # reset counts
        wordCount = 0
        spamCount = 0
        notCount = 0
        # loop over all rows
        for row in training.iterrows(): # how many rows ?
            # if word is in the row
            if training[row, col] > 0:
                # update word count
                wordCount = wordCount + 1
                # if it's spam, add to spam count
                if training[row, 57] == 1:
                    spamCount = spamCount + 1
                # else, add to not spam count
                else:
                    notCount = notCount + 1
        # after looping over all rows
        # do some division. or whatever
        # add to dicts
        pSpam[col] = spamCount / wordCount
        pNot[col] = notCount / wordCount
    # end dataset loop

    # calculate overall probability that something is spam

    totalSpam = 0
    totalNot = 0

    # loop over the rows, count how many are spam and how many aren't
    # do some division or whatver
    for row in training.iterrows():
        if training[row, 57] == 1:
            totalSpam = totalSpam + 1
        else:
            totalNot = totalNot + 1

    prob = dict()

    # for each email in test
    for row in training.iterrows():
        emailSpam = totalSpam
        emailNot = totalNot

        # for each word...
        for col in range(54):
            # if it has the word
            if training[row, col] > 0:
                # multiply probability that word = spam by probability of spam
                emailSpam = emailSpam * pSpam[col]
                # multiply probability that word != spam by probability of not spam
                emailNot = emailNot * pNot[col]
        # whichever probability is higher. say it is that.
        if emailSpam > emailNot:
            prob[col] = 1
        else:
            prob[col] = 0

    return prob


In [95]:
def knn(training, test):

    distances = skl.metrics.pairwise_distances(training, test)

    pred = dict()

    for row in range(len(distances)): # that probably doesn't work but yknow
        # fill array with the first val
        close = list()
        closest = distances[row][0]
        # find closest rows...
        for col in range(len(distances[row])):
            if distances[row][col] <= closest:
                closest = distances[row][col]
                close.append(col)
        # just get the 5 closest
        # i am assuming that there are at least 5 that are in the list
        # there probably are. so it's fine.
        close = close[-5:]

        spam = 0
        for i in range(5):
            spam = spam + test[i, 57]
        
        if spam > 5 - spam:
            pred[row] = 1
        else:
            pred[row] = 0
    
    return pred

In [96]:
def lr(data):
    pass

final actual real things

In [ ]:
val, train = loadData()
naiveBayes(train, val)
knn(train, val)